# Analyse a sample of FUAs with service roads

This notebook produces face polygons and computes shape metrics for each of them.

Unlike the 02_measure notebook, this does not exclude service roads from the network before polygonization.

In [ ]:
import warnings

import geopandas
import dask_geopandas
import numpy
import pygeos
import esda.shape as shape

from tqdm import tqdm

In [ ]:
sample = geopandas.read_parquet("../data/sample.parquet")

## Measure shape characteristics

Polygonize the network to get polygons fully enclosed by street network geometry (face polygons) and measure their shape characteristics.

In [ ]:
# Filter warnings about GeoParquet implementation.
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')

# Loop over unique FUA IDs
for fua_id in tqdm(sample.eFUA_ID, total=len(sample)):
    # Read stret network
    roads = geopandas.read_parquet(f"../data/{int(fua_id)}/roads_osm.parquet")

    # Polygonize street network
    polygons = pygeos.polygonize(roads.geometry.array.data)
    
    # Store geometries as a GeoDataFrame
    polygons = geopandas.GeoDataFrame(
        geometry=geopandas.GeoSeries(
            [polygons], crs=roads.crs
        ).explode(ignore_index=True)
    )
    
    if not polygons.is_valid.all():
        polygons = geopandas.GeoDataFrame(
        geometry=geopandas.GeoSeries(
            pygeos.make_valid(polygons.geometry.values.data), crs=roads.crs
        ).explode(ignore_index=True)
    )

    # Extract underlying PyGEOS geometries pygeos understands (minimum_bounding_circle is not yet exposed in geopandas)
    ga = polygons.geometry.array.data

    # measure area
    polygons["area"] = polygons.area
    # generate minimum bounding circle
    circles = geopandas.GeoDataFrame(
        geometry=geopandas.GeoSeries(pygeos.minimum_bounding_circle(ga), crs=polygons.crs)
    )
    # measure MBC's area
    circles["circle_area"] = circles.area

    # measure Reock (circular) compactness
    polygons["reock"] = polygons["area"] / circles["circle_area"]
  
    # measure direct shape index that captures banana-like relationship between area and Reock compactness in a one dimension
    polygons["shape_index"] = polygons["area"] / numpy.sqrt(circles["circle_area"])
    
    # isoperimetric_quotient
    polygons["isoperimetric_quotient"] = shape.isoperimetric_quotient(polygons)

    # isoareal_quotient
    polygons["isoareal_quotient"] = shape.isoareal_quotient(polygons)

    # radii_ratio
    polygons["radii_ratio"] = shape.radii_ratio(polygons)

    # diameter_ratio
    polygons["diameter_ratio"] = shape.diameter_ratio(polygons)

    # boundary_amplitude
    polygons["boundary_amplitude"] = shape.boundary_amplitude(polygons)

    # squareness
    polygons["squareness"] = shape.squareness(polygons)

    # Schumm's shape_index
    polygons["schumm_shape_index"] = shape.shape_index(polygons)

    # save polygons to a partitioned GeoParquet
    polygons = dask_geopandas.from_geopandas(polygons, npartitions=10)
    polygons.to_parquet(f"../data/{int(fua_id)}/polygons_incl_sr/")
    
    # save circles to a partitioned GeoParquet
    circles = dask_geopandas.from_geopandas(circles, npartitions=10)
    circles.to_parquet(f"../data/{int(fua_id)}/circles_incl_sr/")

  0%|          | 0/131 [00:00<?, ?it/s]/Users/martin/mambaforge/envs/stable/lib/python3.10/site-packages/pygeos/constructive.py:579: RuntimeWarning: invalid value encountered in polygonize
  return lib.polygonize(geometries, **kwargs)
/Users/martin/mambaforge/envs/stable/lib/python3.10/site-packages/pygeos/constructive.py:478: RuntimeWarning: invalid value encountered in make_valid
  return lib.make_valid(geometry, **kwargs)
  2%|▏         | 2/131 [00:00<00:40,  3.16it/s]/Users/martin/mambaforge/envs/stable/lib/python3.10/site-packages/pygeos/constructive.py:579: RuntimeWarning: invalid value encountered in polygonize
  return lib.polygonize(geometries, **kwargs)
  2%|▏         | 3/131 [00:00<00:35,  3.62it/s]/Users/martin/mambaforge/envs/stable/lib/python3.10/site-packages/pygeos/constructive.py:579: RuntimeWarning: invalid value encountered in polygonize
  return lib.polygonize(geometries, **kwargs)
  3%|▎         | 4/131 [00:01<00:29,  4.36it/s]/Users/martin/mambaforge/envs/stable/l